In [1]:
import sys
import pickle
import numpy as np
import torch 

sys.path.append("../src")

import constants as constants
constants.load("../data/moves_10", True)

import neural_net
from neural_net import NeuralNet, ResidualBlock

""

Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners
Number of moves: 6233
Board size: 10


''

In [2]:
model = NeuralNet().to("mps")
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
sum([np.prod(p.size()) for p in model_parameters])

85480681

In [3]:
for _ in range(1000):
    occupancies = np.zeros((4, constants.BOARD_SIZE, constants.BOARD_SIZE), dtype=bool)
    neural_net.evaluate(model, occupancies, "mps")

In [30]:
import torch.nn as nn
import torch.nn.functional as F

In [31]:
nn.CrossEntropyLoss()(
    torch.tensor([[1.0, 5.0, 3.0]]),
    F.softmax(torch.tensor([[1.0, 5.0, 3.0]]), dim=1),
)

tensor(0.4411)

In [32]:
nn.CrossEntropyLoss()(
    torch.tensor([[1.0, 5.0, 3.0]]),
    F.softmax(torch.tensor([[1.0, 5.0, 3.0]]), dim=1),
)

tensor(0.1599)